In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import seaborn as sns
from matplotlib import pyplot as plt
from scipy.stats import norm
from utils.params import fig_dir
%config InlineBackend.figure_format ='retina'
mpl.style.use("ggplot")

In [ ]:
df = pd.read_csv("data/global-plastics-production.csv")
df["plastic_production"] = (
    df["Global plastics production (million tonnes)"] / 1_000_000
)

fig, ax = plt.subplots()
fig.set_figheight(5)
fig.set_figwidth(7.5)

plt.plot(df["Year"], df["plastic_production"], color="#2E3192")
ax.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter("{x:,.0f}M"))
plt.xlabel("Year")
plt.ylabel("Plastic Production [t]")

plt.savefig(
    fig_dir + "/plastic_production.png",
    dpi=300,
    bbox_inches="tight",
    pad_inches=0,
)

In [ ]:
df = pd.read_csv("data/global-plastic-fate.csv")

df_discarded = df[df["Entity"] == "Discarded"]
df_discarded["percentage"] = (
    df_discarded["Estimated historic plastic fate"] / 100
)
df_discarded = df_discarded[["Year", "percentage"]]

df_incinerated = df[df["Entity"] == "Incinerated"]
df_incinerated["percentage"] = (
    df_incinerated["Estimated historic plastic fate"] / 100
)
df_incinerated = df_incinerated[["Year", "percentage"]]

df_recycled = df[df["Entity"] == "Recycled"]
df_recycled["percentage"] = df_recycled["Estimated historic plastic fate"] / 100
df_recycled = df_recycled[["Year", "percentage"]]

In [ ]:
fig, ax = plt.subplots()
fig.set_figheight(4.4)
fig.set_figwidth(6.7)

plt.stackplot(
    df_discarded["Year"],
    df_discarded["percentage"],
    df_incinerated["percentage"],
    df_recycled["percentage"],
    colors=[plt.cm.tab10(0), plt.cm.tab10(1), plt.cm.tab10(2)],
    alpha=0.65,
)

plt.annotate(
    "Discarded",
    xy=(1.01, 0.25),
    xycoords="axes fraction",
    color=plt.cm.tab10(0),
)
plt.annotate(
    "Incinerated",
    xy=(1.01, 0.665),
    xycoords="axes fraction",
    color=plt.cm.tab10(1),
)
plt.annotate(
    "Recycled", xy=(1.01, 0.89), xycoords="axes fraction", color=plt.cm.tab10(2)
)

ax.yaxis.set_major_formatter(mpl.ticker.PercentFormatter(xmax=1))
plt.xlabel("Year")
ax.set_facecolor("white")
ax.set_xmargin(0)
ax.set_ymargin(0)

plt.savefig(
    fig_dir + "/plastic_fate.png", dpi=300, bbox_inches="tight", pad_inches=0
)

In [ ]:
def skew_norm_pdf(x, e=0, w=1, a=0):
    t = (x - e) / w
    return 2.0 * w * norm.pdf(t) * norm.cdf(a * t)


fig = plt.figure()
fig.set_figheight(4)
fig.set_figwidth(6)

plt.plot(
    skew_norm_pdf(np.linspace(-1, 3, 100), 0, 1, 3), zorder=10, color="#2E3192"
)

plt.axvline(
    x=34, ymin=0.05, ymax=0.94, color="black", linestyle="--", linewidth=1
)
plt.annotate("$M_n$", xy=(35, 0.25), va="center", size=12)
plt.axvline(
    x=45, ymin=0.05, ymax=0.83, color="black", linestyle="--", linewidth=1
)
plt.annotate("$M_w$", xy=(46, 0.25), va="center", size=12)

ax = plt.gca()
ax.xaxis.set_ticks([])
ax.yaxis.set_ticks([])
plt.xlabel("$log$(Molecular Weight)", size=11)
plt.ylabel("Amount / Frequency", size=11)

plt.savefig(
    fig_dir + "/mw_distribution.png", dpi=300, bbox_inches="tight", pad_inches=0
)

In [ ]:
np.random.seed(42)
X1 = np.random.normal(1, 0.3, 10000)
X2 = np.random.normal(2, 0.38, 10000)
X = np.concatenate([X1, X2])

fig, ax = plt.subplots()
fig.set_figheight(4)
fig.set_figwidth(6)

sns.kdeplot(X, bw_adjust=2, ax=ax, color="#2E3192")

plt.axvline(
    x=1.12, ymin=0.05, ymax=0.929, color="black", linestyle="--", linewidth=1
)
plt.annotate("$M_w^s$", xy=(1.16, 0.25), va="center", size=12)
plt.axvline(
    x=2.07, ymin=0.05, ymax=0.759, color="black", linestyle="--", linewidth=1
)
plt.annotate("$M_w^l$", xy=(2.11, 0.25), va="center", size=12)

plt.ylim((-0.03, 0.643))
plt.xlim((-0.1, 3.2))
ax = plt.gca()
ax.xaxis.set_ticks([])
ax.yaxis.set_ticks([])
plt.xlabel("$log$(Molecular Weight)", color="#595959", size=11)
plt.ylabel("Amount / Frequency", color="#595959", size=11)
ax.set_facecolor("#E5E5E5")

plt.savefig(
    fig_dir + "/bimodal_mw_dist.png", dpi=300, bbox_inches="tight", pad_inches=0
)

In [ ]:
data = []
with open("data/RunPSPDI/gtp_Z200_PDI5.0.dat") as f:
    for line in f.readlines():
        data.append(line.split())

df = pd.DataFrame(data, dtype=float)
df.columns = ["freq", "G'", "G''"]

fig, ax = plt.subplots()
fig.set_figheight(5)
fig.set_figwidth(7.5)

plt.loglog(
    df["freq"],
    df["G'"],
    base=10,
    zorder=10,
    color=plt.cm.tab10(0),
    label="Storage modulus ($G'$)",
)
plt.loglog(
    df["freq"],
    df["G''"],
    base=10,
    zorder=10,
    color=plt.cm.tab10(1),
    label="Loss modulus ($G''$)",
)

plt.minorticks_off()
plt.xlabel("Frequency [$s^{-1}$]")
plt.ylabel("$G'$, $G''$ [Pa]")
plt.legend(loc="lower right", framealpha=1, edgecolor="None")

plt.savefig(fig_dir + "/moduli.png", dpi=300, bbox_inches="tight", pad_inches=0)